## Analista de Datos enerBit

In [1]:
import pandas as pd
import numpy as np
import os
import plotly.graph_objects as go

current_dir = os.getcwd()

In [2]:
# Se contruye las rutas relativas a los archivos en la carpeta Datasets.

datasets_dir = os.path.join(current_dir, '..', 'Datasets')
certificados_path = os.path.join(datasets_dir, 'certificados_v3.xlsx')
estados_path = os.path.join(datasets_dir, 'estados_y_seriales_v2.xlsx')
critica_path = os.path.join(datasets_dir, 'resumen_criticas_v2.xlsx')

Este informe aborda el análisis de tres conjuntos de datos:

- Estados y Seriales: Proporciona información sobre la ubicación, incluyendo dirección, municipio, ciudad y serie del medidor.
- Resumen de Críticas: Muestra los intentos realizados diariamente y los resultados de las críticas de medida para cada medidor, clasificando los resultados en Verdadero o Falso.
- Datos de Certificados: Contiene detalles sobre las órdenes de trabajo, series de medidores y tipos de órdenes de trabajo operativas a realizar.

In [3]:
# Se lee los archivos en excel y se guargan como Dataframes
df_estado = pd.read_excel(estados_path)
df_critica = pd.read_excel(critica_path)
df_certificados = pd.read_excel(certificados_path)

### Analisis Exploratorio (EDA)

In [4]:
#Se mira la cabecera de los archivos
print("\nEstado:")
print(df_estado.head())

print("\nCrítica:")
print(df_critica.head())


print("Certificados:")
print(df_certificados.head())




Estado:
                address      state          city      serial
0  Carrera 32 Calle 100  Santander        Ibagué  139955a8a8
1   Carrera 42 Calle 79    Quindío      Medellín  c77f155c50
2   Carrera 23 Calle 70    Bolívar  Barranquilla  ffed9e7ad8
3   Carrera 43 Calle 63    Quindío        Bogotá  07ad945e44
4  Carrera 53 Calle 100      Cesar     Cartagena  0e79a243ee

Crítica:
                 desde                hasta                 tipo  subjuicio  \
0  2023-12-09 00:00:00  2023-12-10 00:00:00  ALL-PERIODS-PRESENT      False   
1  2023-12-09 00:00:00  2023-12-10 00:00:00  ALL-PERIODS-PRESENT      False   
2  2023-12-09 00:00:00  2023-12-10 00:00:00  ALL-PERIODS-PRESENT      False   
3  2023-12-09 00:00:00  2023-12-10 00:00:00  ALL-PERIODS-PRESENT      False   
4  2023-12-09 00:00:00  2023-12-10 00:00:00  ALL-PERIODS-PRESENT      False   

   periodos_encontrados  periodos_solicitados      serial  
0                    12                    24  139955a8a8  
1                   

In [5]:
print("\nEstado Info:")
df_estado.info()

print("\nCrítica Info:")
df_critica.info()

print("Certificados Info:")
df_certificados.info()



Estado Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 977 entries, 0 to 976
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   address  977 non-null    object
 1   state    977 non-null    object
 2   city     977 non-null    object
 3   serial   977 non-null    object
dtypes: object(4)
memory usage: 30.7+ KB

Crítica Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 185963 entries, 0 to 185962
Data columns (total 7 columns):
 #   Column                Non-Null Count   Dtype 
---  ------                --------------   ----- 
 0   desde                 185963 non-null  object
 1   hasta                 185963 non-null  object
 2   tipo                  185963 non-null  object
 3   subjuicio             185963 non-null  bool  
 4   periodos_encontrados  185963 non-null  int64 
 5   periodos_solicitados  185963 non-null  int64 
 6   serial                185963 non-null  object
dtypes: bool(1), int64(2), object(4

In [6]:
#Explorar valores unicos y distribucion de las variables 
print("Certificados Unique Values:")
for col in df_certificados.select_dtypes(include='object').columns:
    print(f"{col}: {df_certificados[col].nunique()} unique values")

print("\nEstado Unique Values:")
for col in df_estado.select_dtypes(include='object').columns:
    print(f"{col}: {df_estado[col].nunique()} unique values")

print("\nCrítica Unique Values:")
for col in df_critica.select_dtypes(include='object').columns:
    print(f"{col}: {df_critica[col].nunique()} unique values")


Certificados Unique Values:
id: 1861 unique values
created_at: 1861 unique values
order_id: 318 unique values
certificate_type: 3 unique values
serial: 975 unique values

Estado Unique Values:
address: 926 unique values
state: 14 unique values
city: 14 unique values
serial: 975 unique values

Crítica Unique Values:
desde: 134 unique values
hasta: 134 unique values
tipo: 1 unique values
serial: 975 unique values


**Conjunto de datos "Estados y seriales"**

Este conjunto de datos proporciona la ubicación de los medidores instalados. Es importante destacar que cada medidor tiene un serial único.

Para este conjunto, se identificaron dos series de medidores que requieren una limpieza debido a la presencia de duplicados.

In [7]:
# Mostrar todas las filas duplicadas
duplicated_rows = df_estado[df_estado.duplicated()]

print("Filas duplicadas en df_estado:")
print(duplicated_rows)


Filas duplicadas en df_estado:
                 address      state          city      serial
844  Carrera 18 Calle 36  Risaralda       Pereira  f57f243205
845  Carrera 29 Calle 80  Magdalena  Barranquilla  9fba6de76a


In [8]:
# Eliminar duplicados en el dataframe original y actualizarlo
df_estado.drop_duplicates(inplace=True)


total_medidores = df_estado['serial'].nunique()

print(total_medidores)

975


Del análisis realizado, se identificaron un total de 975 medidores distribuidos en diversas ciudades. Las ciudades con la mayor cantidad de medidores instalados son Cartagena, Santa Marta y Cali. El promedio de medidores instalados por ciudad es de 69.

In [9]:


# Contar la frecuencia de medidores por ciudad
city_counts = df_estado['city'].value_counts()

# Calcular el promedio de medidores instalados por ciudad
average_medidores = city_counts.mean()

# Crear el gráfico de barras con Plotly
fig = go.Figure(data=[
    go.Bar(
        name='Cantidad de Medidores', 
        x=city_counts.index, 
        y=city_counts.values, 
        marker=dict(color='#8338ec'),
        text=city_counts.values,
        textposition='inside',
        textfont=dict(color='white')  # Color del texto dentro de las barras
    )
])

#línea horizontal que represente el promedio de medidores
fig.add_trace(go.Scatter(
    x=city_counts.index,
    y=[average_medidores] * len(city_counts),
    mode='lines',
    name='Promedio medidores instalados 69',
    line=dict(color='red', dash='dash')
))

# Ajustar el diseño y formato del gráfico
fig.update_layout(
    title='Cantidad de Medidores por Ciudad',
    xaxis_title='Ciudad',
    yaxis_title='Cantidad de Medidores',
    font=dict(size=11),
    template='plotly_white'
)

# Mostrar el gráfico
fig.show()


In [10]:
import pandas as pd
import plotly.graph_objects as go


# Agrupar por 'state' y 'city' y contar la cantidad de 'serial' para cada combinación
grouped = df_estado.groupby(['state', 'city']).size().reset_index(name='count')

# Crear la tabla pivotada
pivot_table = grouped.pivot_table(index='state', columns='city', values='count', fill_value=0, aggfunc='sum')

# Agregar una fila de totales
pivot_table.loc['Total'] = pivot_table.sum()

# Crear la tabla en Plotly
fig = go.Figure(data=[go.Table(
    header=dict(values=['City'] + list(pivot_table.columns),
                fill_color='paleturquoise',
                align='left'),
    cells=dict(values=[pivot_table.index] + [pivot_table[col] for col in pivot_table.columns],
               fill_color='lavender',
               align='left'))
])

# Ajustar el diseño y formato
fig.update_layout(
    title='Cantidad de Medidores por Departamento y Ciudad',
    font=dict(size=11),  # Ajustar el tamaño del texto
    template='plotly_white'  # Cambiar el tema a plotly_white
)

# Mostrar la tabla
fig.show()


Se observa que los registros de ubicación están incorrectamente asociados, ya que hay ciudades en departamentos que no corresponden. Por lo tanto, es necesario reforzar la captura de información, asegurándose de que se ingresen correctamente tanto el departamento como la ciudad.

**Conjunto de Datos resumen_criticas**

In [11]:
# Contar la frecuencia de medidores por ciudad
subjuicio_counts = df_critica['subjuicio'].value_counts()

# Crear el gráfico de barras con Plotly
fig = go.Figure(data=[
    go.Bar(
        name='Cantidad de Medidores', 
        x=subjuicio_counts.index, 
        y=subjuicio_counts.values, 
        marker=dict(color='#8338ec'),
        text=subjuicio_counts.values,
        textposition='inside',
        textfont=dict(color='white')  # Color del texto dentro de las barras
    )
])

# Ajustar el diseño y formato del gráfico
fig.update_layout(
    title='Cantidad de Datos de Telemedidad',
    xaxis_title='Telemedida completa',
    yaxis_title='Datos completos de Telemedidad',
    font=dict(size=11),
    template='plotly_white'
)

# Mostrar el gráfico
fig.show()

In [12]:
# Convertir la columna 'desde' a datetime
df_critica['desde'] = pd.to_datetime(df_critica['desde'])

# Extraer el mes de la columna 'desde'
df_critica['mes'] = df_critica['desde'].dt.month

# Mapa de los meses
meses = {1: 'Enero', 2: 'Febrero', 3: 'Marzo', 4: 'Abril', 5: 'Mayo', 6: 'Junio', 7: 'Julio', 8: 'Agosto', 9: 'Septiembre', 10: 'Octubre', 11: 'Noviembre', 12: 'Diciembre'}
df_critica['mes_nombre'] = df_critica['mes'].map(meses)

# Agrupar por mes y subjuicio, y contar la cantidad de registros
grupo_mes_subjuicio = df_critica.groupby(['mes_nombre', 'subjuicio']).size().unstack(fill_value=0).reset_index()

# Reordenar los meses
orden_meses = ['Enero', 'Febrero', 'Marzo', 'Abril', 'Mayo', 'Junio', 'Julio', 'Agosto', 'Septiembre', 'Octubre', 'Noviembre', 'Diciembre']
grupo_mes_subjuicio['mes_nombre'] = pd.Categorical(grupo_mes_subjuicio['mes_nombre'], categories=orden_meses, ordered=True)
grupo_mes_subjuicio = grupo_mes_subjuicio.sort_values('mes_nombre')

# Crear el gráfico de barras
fig = go.Figure()

colors = {True: '#FF7705', False: '#8338ec'}

for subjuicio in [True, False]:
    fig.add_trace(go.Bar(
        x=grupo_mes_subjuicio['mes_nombre'],
        y=grupo_mes_subjuicio[subjuicio],
        name=f'Telemedida exitosa: {subjuicio}',
        marker_color=colors[subjuicio],
        text=grupo_mes_subjuicio[subjuicio],
        textposition='auto'
    ))

# Ajustar el diseño y formato del gráfico
fig.update_layout(
    barmode='group',
    title='Cantidad de intentos de Telemedida por Mes',
    xaxis_title='Mes',
    yaxis_title='Cantidad de Intentos',
    font=dict(size=11),
    template='plotly_white'
)

# Mostrar el gráfico
fig.show()


En el mes de abril se realizaron más críticas de medida en comparación con otros meses. Sería conveniente profundizar y evaluar si hubo factores externos que afectaron la telemedida, como la temporada de lluvias, problemas con el operador de red, entre otros.

In [13]:
# Filtrar los registros donde 'subjuicio' es False
df_false = df_critica[df_critica['subjuicio'] == False]

# Contar la frecuencia de medidores por serial
serial_counts = df_false['serial'].value_counts()

# Seleccionar los 20 mayores
top_20_serial_counts = serial_counts.nlargest(20)

# Crear el gráfico de barras con Plotly
fig = go.Figure(data=[
    go.Bar(
        name='Cantidad de Medidores', 
        x=top_20_serial_counts.index, 
        y=top_20_serial_counts.values, 
        marker=dict(color='#8338ec'),
        text=top_20_serial_counts.values,
        textposition='inside',
        textfont=dict(color='white')  # Color del texto dentro de las barras
    )
])

# Ajustar el diseño y formato del gráfico
fig.update_layout(
    title='Top 20 medidores con mas intentos de Telemedida',
    xaxis_title='Serial',
    yaxis_title='Cantidad de intentos de comunicación',
    font=dict(size=11),
    template='plotly_white'
)

# Mostrar el gráfico
fig.show()


In [14]:
# Realizar la combinación de los DataFrames basándonos en la columna 'serial'
df_dos = df_critica.merge(df_estado[['serial', 'city']], on='serial', how='left')

# Mostrar el DataFrame resultante
print(df_dos)

            desde                hasta                 tipo  subjuicio  \
0      2023-12-09  2023-12-10 00:00:00  ALL-PERIODS-PRESENT      False   
1      2023-12-09  2023-12-10 00:00:00  ALL-PERIODS-PRESENT      False   
2      2023-12-09  2023-12-10 00:00:00  ALL-PERIODS-PRESENT      False   
3      2023-12-09  2023-12-10 00:00:00  ALL-PERIODS-PRESENT      False   
4      2023-12-09  2023-12-10 00:00:00  ALL-PERIODS-PRESENT      False   
...           ...                  ...                  ...        ...   
185958 2024-06-12  2024-06-13 00:00:00  ALL-PERIODS-PRESENT       True   
185959 2024-06-12  2024-06-13 00:00:00  ALL-PERIODS-PRESENT       True   
185960 2024-06-12  2024-06-13 00:00:00  ALL-PERIODS-PRESENT       True   
185961 2024-06-12  2024-06-13 00:00:00  ALL-PERIODS-PRESENT      False   
185962 2024-06-12  2024-06-13 00:00:00  ALL-PERIODS-PRESENT       True   

        periodos_encontrados  periodos_solicitados      serial  mes  \
0                         12            

In [15]:
# Contar la frecuencia de medidores por ciudad
telemedida_counts = df_dos['city'].value_counts()

# Crear el gráfico de barras con Plotly
fig = go.Figure(data=[
    go.Bar(
        name='Cantidad de Medidores', 
        x=telemedida_counts.index, 
        y=telemedida_counts.values, 
        marker=dict(color='#FF7705'),
        text=telemedida_counts.values,
        textposition='inside',
        textfont=dict(color='white')  # Color del texto dentro de las barras
    )
])

# Ajustar el diseño y formato del gráfico
fig.update_layout(
    title='LLamadas no exitosas de Telemedidad  por Ciudad',
    xaxis_title='Ciudad',
    yaxis_title='Cantidad de Intentos de Telemedida',
    font=dict(size=11),
    template='plotly_white'
)

# Mostrar el gráfico
fig.show()

Se observa que las ciudades donde se realizan más intentos de "Crítica de medida" o telemedida son Cartagena, Valledupar, y Santa Marta. La cantidad de intentos realizados a los medidores hasta que la telemedida sea exitosa puede depender del tipo de tecnología utilizada, los medios de comunicación y la cobertura del operador. Por lo tanto, sería interesante contar con esta información para categorizar los datos y establecer planes de mantenimiento que ayuden a mejorar la telemedida.

**Conjunto de Datos Datos certificados**

In [20]:
# Contar la frecuencia de 'serial' para cada 'certificate_type'
frequency_table = df_certificados.groupby('certificate_type')['serial'].count().reset_index()
frequency_table.columns = ['Tipo de Maniobra', 'Cantidad']

# Calcular la altura del gráfico basado en el número de filas
num_rows = len(frequency_table)
table_height = max(100, 40 * num_rows + 40)  # 30 pixels per row + 40 pixels for padding

# Crear la tabla en Plotly
fig = go.Figure(data=[go.Table(
    header=dict(values=list(frequency_table.columns),
                fill_color='paleturquoise',
                align='left'),
    cells=dict(values=[frequency_table[col] for col in frequency_table.columns],
               fill_color='lavender',
               align='left'))
])

# Ajustar el diseño y formato del gráfico
fig.update_layout(
    title='Cantidad De Actividades Realizadas',
    font=dict(size=11),
    template='plotly_white',
    height=table_height,  # Ajustar la altura del gráfico
    margin=dict(l=10, r=10, t=40, b=10)  # Reducir márgenes
)

# Mostrar el gráfico
fig.show()


In [25]:
# Contar la frecuencia de cada 'serial'
serial_counts = df_certificados['serial'].value_counts()

# Filtrar seriales que aparecen exactamente una vez
unique_serials = serial_counts[serial_counts == 1].index

# Filtrar el DataFrame original para mantener solo los seriales únicos
df_unique_serials = df_certificados[df_certificados['serial'].isin(unique_serials)]

# Contar la frecuencia de seriales únicos para cada 'certificate_type'
frequency_table = df_unique_serials.groupby('certificate_type')['serial'].count().reset_index()
frequency_table.columns = ['Certificate Type', 'Unique Serial Count']

# Calcular la altura del gráfico basado en el número de filas
num_rows = len(frequency_table)
table_height = max(100, 40 * num_rows + 40)  # 40 pixels per row + 40 pixels for padding

# Crear la tabla en Plotly
fig = go.Figure(data=[go.Table(
    header=dict(values=['Certificate Type', 'Unique Serial Count'],
                fill_color='paleturquoise',
                align='left'),
    cells=dict(values=[frequency_table['Certificate Type'], frequency_table['Unique Serial Count']],
               fill_color='lavender',
               align='left'))
])

# Ajustar el diseño y formato del gráfico
fig.update_layout(
    title='Cantidad de Actividades sin Modem asociado',
    font=dict(size=11),  # Ajustar el tamaño del texto
    template='plotly_white',  # Cambiar el tema a plotly_white
    height=table_height,  # Ajustar la altura del gráfico
    margin=dict(l=10, r=10, t=40, b=10)  # Reducir márgenes
)

# Mostrar el gráfico
fig.show()


De los datos analizados se observa que a cada tipo de serie de medidor único se le instala un medidor y un módem. Se tiene:

- 980 medidores instalados.
- 881 módems instalados.

Por lo tanto, aún están pendientes de instalación 99 módems que están en espera por orden de trabajo ("Orden_id").


In [19]:
# Convertir la columna 'created_at' a datetime
df_certificados['created_at'] = pd.to_datetime(df_certificados['created_at'])

# Extraer el mes de la columna 'created_at'
df_certificados['mes'] = df_certificados['created_at'].dt.month

# Mapa de los meses
meses = {1: 'Enero', 2: 'Febrero', 3: 'Marzo', 4: 'Abril', 5: 'Mayo', 6: 'Junio', 7: 'Julio', 8: 'Agosto', 9: 'Septiembre', 10: 'Octubre', 11: 'Noviembre', 12: 'Diciembre'}
df_certificados['mes_nombre'] = df_certificados['mes'].map(meses)

# Agrupar por mes y certificate_type, y contar la cantidad de seriales
grupo_mes_certificado = df_certificados.groupby(['mes_nombre', 'certificate_type']).size().unstack(fill_value=0).reset_index()

# Reordenar los meses
orden_meses = ['Enero', 'Febrero', 'Marzo', 'Abril', 'Mayo', 'Junio', 'Julio', 'Agosto', 'Septiembre', 'Octubre', 'Noviembre', 'Diciembre']
grupo_mes_certificado['mes_nombre'] = pd.Categorical(grupo_mes_certificado['mes_nombre'], categories=orden_meses, ordered=True)
grupo_mes_certificado = grupo_mes_certificado.sort_values('mes_nombre')

# Crear el gráfico de barras
fig = go.Figure()

for certificate_type in grupo_mes_certificado.columns[1:]:
    fig.add_trace(go.Bar(
        x=grupo_mes_certificado['mes_nombre'],
        y=grupo_mes_certificado[certificate_type],
        name=certificate_type,
        text=grupo_mes_certificado[certificate_type],
        textposition='auto'
    ))

# Ajustar el diseño y formato del gráfico
fig.update_layout(
    barmode='group',
    title='Cantidad Tipo Actividad Por Mes',
    xaxis_title='Mes',
    yaxis_title='Cantidad de Medidores',
    font=dict(size=11),
    template='plotly_white'
)

# Mostrar el gráfico
fig.show()
